<a href="https://colab.research.google.com/github/CharlesOkafor91/cat_vs_dog/blob/master/dog_vs_cat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# importing the necessary libraries to start

import torch
from torch import nn, optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.autograd import Variable

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import os, random

In [0]:
# Define transform to help transform the image data for both train and test

train_transform = transforms.Compose([transforms.RandomRotation(30),
                                     transforms.RandomResizedCrop(100),
                                     transforms.RandomHorizontalFlip(),
                                     transforms.ToTensor(),
                                     transforms.Normalize([0.485, 0.456, 0.406],
                                                         [0.229, 0.224, 0.225])])

test_transform = transforms.Compose([transforms.Resize(255),
                                     transforms.CenterCrop(224),
                                     transforms.ToTensor()])

In [3]:
# connecting my google drive to get data and save my model

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
# setting my working path to my google drive
path = "/content/gdrive/My Drive/Colab Notebooks/deep_learning_challenge/Cat_Dog_data"
os.chdir(path)
!ls

fc_model.py  helper.py	test  train


In [6]:
# loading data with datasets.ImageFolder from our drive path

train_data = datasets.ImageFolder(path + '/train', transform=train_transform)
test_data = datasets.ImageFolder(path + '/test', transform=test_transform)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=True)
train_data

Dataset ImageFolder
    Number of datapoints: 22507
    Root location: /content/gdrive/My Drive/Colab Notebooks/deep_learning_challenge/Cat_Dog_data/train
    StandardTransform
Transform: Compose(
               RandomRotation(degrees=(-30, 30), resample=False, expand=False)
               RandomResizedCrop(size=(100, 100), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=PIL.Image.BILINEAR)
               RandomHorizontalFlip(p=0.5)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

In [10]:
# here we define the device we want to work on. Cuda is we have gpu else device will be cpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [12]:
# we will be usng pre-trained model (densenet121) from ImageNet. This is called transfered learning 

model = model = models.resnet50(pretrained=True)
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [13]:
# Now our pre-trained model above has both features and classifier.
# we need to re-train the claffier and not the features. This is because the classifier (fc)...
# which is a linear transformation of in features 2048 and out features of 1000 in the pre-trained model is not well tailored for our task
# for example it has 1000 out features whereas our model needs to ouput just 2 classes (dog or cat)
# first we freeze the features and then re-train the classifier

#turn off gradients for our model
for param in model.parameters():
  param.requires_grad = False

#defining our new classifier to replace the classifier in the pre-trained model
classifier = nn.Sequential(
    nn.Linear(2048, 512),
    nn.ReLU(),
    nn.Dropout(p = 0.2),
    nn.Linear(512, 2),
    nn.LogSoftmax(dim=1)
)

model.fc = classifier # replacing the classifier with our newly defined

criterion = nn.NLLLoss() # defining our loss object using the negative log likelihood loss function

optimizer = optim.Adam(model.fc.parameters(), lr=0.003) # using the Adam optimizer from optim module to define our optimizer 

model.to(device) # setting our model to the device we are using (cuda for gpu or cpu)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [15]:
# now we have moved our model to our gpu, we would run a little experiment to show gpu is fater than cpu in process neural networks
# all we did here is to test to see that gpu is faster than cpu

import time

for cuda in (False, True): 

  criterion = nn.NLLLoss()

  optimizer = optim.Adam(model.fc.parameters(), lr=0.001) 

  if cuda:
    model.cuda()
  else:
    model.cpu() # setting our model back to cpu device for when cuda is false. We are not using device here as we do not want to change the global definition of device
  
  for ii, (inputs, labels) in enumerate(trainloader):
    inputs, labels = Variable(inputs), Variable(labels)

    if cuda:
      inputs, labels = inputs.cuda(), labels.cuda()

    start = time.time()

    outputs = model.forward(inputs)
    loss =  criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    if ii == 3:
      break
  print(f"CUDA = {cuda}; Time Per Batch: {(time.time() - start)/3:.3f} seconds")

CUDA = False; Time Per Batch: 0.935 seconds
CUDA = True; Time Per Batch: 0.004 seconds


In [0]:
# From the above, we can see that the network trains faster with cuda

# now we train our model (classifer) with our dataset
epochs = 1
steps = 0
running_loss = 0
print_every = 5

for e in range(epochs):
  
  for images, labels in trainloader:
    steps += 1

    #move our images and labels to the gpu
    images, labels = images.to(device), labels.to(device)

    optimizer.zero_grad()

    logps = model(images)
    loss = criterion(logps, labels)
    loss.backward()
    optimizer.step()

    #increment our running loss
    running_loss += loss.item()

    #to test with our test data at the print_every intervals
    if steps % print_every == 0:
      model.eval() # to kill dropout in our test model
      test_loss = 0
      accuracy = 0

      for images, labels in testloader:

        images, labels = images.to(device), labels.to(device)

        logets = model(images)
        loss = criterion(logets, labels)
        test_loss += loss.item()

        #calculate our accuracy
        ps = torch.exp(logets)
        top_ps, top_class = ps.topk(1, dim=1)
        equality = top_class == labels.view(*top_class.shape)
        accuracy += torch.mean(equality.type(torch.FloatTensor)).item()
      
      print(f"Epoch {e+1}/{epochs}... "
      f"Training Loss: {running_loss/print_every:.3f}... "
      f"Test Loss: {test_loss/len(testloader):.3f}... "
      f"Test Accuracy: {accuracy/len(testloader):.3f}"
      )

      running_loss = 0
      model.train()


Epoch 1/1... Training Loss: 0.313... Test Loss: 0.233... Test Accuracy: 0.910
Epoch 1/1... Training Loss: 0.324... Test Loss: 0.356... Test Accuracy: 0.832
Epoch 1/1... Training Loss: 0.320... Test Loss: 0.356... Test Accuracy: 0.860
Epoch 1/1... Training Loss: 0.304... Test Loss: 0.507... Test Accuracy: 0.713
Epoch 1/1... Training Loss: 0.369... Test Loss: 0.407... Test Accuracy: 0.846
Epoch 1/1... Training Loss: 0.292... Test Loss: 0.413... Test Accuracy: 0.845
Epoch 1/1... Training Loss: 0.280... Test Loss: 0.581... Test Accuracy: 0.664
Epoch 1/1... Training Loss: 0.292... Test Loss: 0.423... Test Accuracy: 0.836
Epoch 1/1... Training Loss: 0.268... Test Loss: 0.478... Test Accuracy: 0.744
Epoch 1/1... Training Loss: 0.304... Test Loss: 0.458... Test Accuracy: 0.765
Epoch 1/1... Training Loss: 0.330... Test Loss: 0.566... Test Accuracy: 0.678
Epoch 1/1... Training Loss: 0.246... Test Loss: 0.526... Test Accuracy: 0.700
Epoch 1/1... Training Loss: 0.251... Test Loss: 0.561... Test Ac